<a href="https://colab.research.google.com/github/DJCordhose/ai/blob/master/notebooks/tensorflow/tf_low_level_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Low Level TensorFlow, Part II: Advanced

* https://www.tensorflow.org/guide/low_level_intro

In [1]:
# import and check version
import tensorflow as tf
# tf can be really verbose
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.12.0


In [2]:
# a small sanity check, does tf seem to work ok? 
hello = tf.constant('Hello TF!')
sess = tf.Session()
print(sess.run(hello))
sess.close()

b'Hello TF!'


## Reading in data sets

In [3]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

r = tf.random_normal([10, 2])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

with tf.Session() as sess:
  sess.run(iterator.initializer)
  while True:
    try:
      data = sess.run(next_row)
      print(data)
      print(sess.run(z, feed_dict={x: data[0], y: data[1]}))
    except tf.errors.OutOfRangeError:
      break

[-0.09776118  0.4570114 ]
0.35925022
[ 0.50097823 -1.0027725 ]
-0.5017942
[1.7740805  0.18853252]
1.962613
[-1.4601867 -0.0258622]
-1.4860489
[-0.32546598  0.56940705]
0.24394107
[ 0.5967923 -1.2927308]
-0.6959385
[-1.7093807  1.1030443]
-0.6063365
[-1.4031861  1.2597154]
-0.14347064
[-0.577856   -0.27080232]
-0.8486583
[ 0.812819   -0.02773912]
0.7850799


## Layers

In [4]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(inputs=x, units=1)

with tf.Session() as sess:
  try:
    print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))
  except tf.errors.FailedPreconditionError as fpe:
    print(fpe.message)

Attempting to use uninitialized value dense/kernel
	 [[node dense/kernel/read (defined at <ipython-input-4-eb68a0e5f9f4>:2)  = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense/kernel)]]
	 [[{{node dense/BiasAdd/_3}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_12_dense/BiasAdd", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]


In [5]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[2.6953735]
 [3.3833814]]


In [6]:
y = tf.layers.dense(inputs=x, units=2, activation=tf.nn.tanh)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[0.78271574 0.8640919 ]
 [0.8852238  0.99992716]]


## Feature columns

transform a diverse range of raw data into formats input layers can accept

* https://www.tensorflow.org/guide/feature_columns
* https://www.tensorflow.org/api_docs/python/tf/feature_column/input_layer

In [0]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']
}

# numeric values are simple
sales_column = tf.feature_column.numeric_column('sales')
columns = {
   sales_column
}

inputs = tf.feature_column.input_layer(features, columns)

In [8]:
# categories are harders, as NNs only accept dense numeric values

categorical_department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])

columns = {
  sales_column,
  categorical_department_column
}

# we can decide if we want the category to be encoded as embedding or multi-hot 
try:
  inputs = tf.feature_column.input_layer(features, columns)
except ValueError as ve:
  print(ve)

Items of feature_columns must be a _DenseColumn. You can wrap a categorical column with an embedding_column or indicator_column. Given: _VocabularyListCategoricalColumn(key='department', vocabulary_list=('sports', 'gardening'), dtype=tf.string, default_value=-1, num_oov_buckets=0)


In [0]:
multi_hot_department_column = tf.feature_column.indicator_column(categorical_department_column)

In [0]:
columns = {
  sales_column,
  multi_hot_department_column
}

inputs = tf.feature_column.input_layer(features, columns)

In [11]:
# feature columns also need initialization
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
with tf.Session() as sess:
  sess.run((var_init, table_init))
  # first two are departments last entry is just sales as is
  print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


In [12]:
# multi (one in our case) hot encoding of departments
columns = {
  multi_hot_department_column
}

inputs = tf.feature_column.input_layer(features, columns)
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
with tf.Session() as sess:
  sess.run((var_init, table_init))
  print(sess.run(inputs))


[[1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [13]:
# alternative, embedding in three dimensions
embedding_department_column = tf.feature_column.embedding_column(categorical_department_column, dimension=3)
columns = {
  embedding_department_column
}

inputs = tf.feature_column.input_layer(features, columns)
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
with tf.Session() as sess:
  sess.run((var_init, table_init))
  print(sess.run(inputs))

[[ 0.87459344  0.22822124 -0.5744384 ]
 [ 0.87459344  0.22822124 -0.5744384 ]
 [-0.09880681  0.13576707  0.10532397]
 [-0.09880681  0.13576707  0.10532397]]
